In [83]:
import requests
import json
import re

files_endpt = "https://api.gdc.cancer.gov/files"

filters = {
    "op": "and",
    "content":[
        {
        "op": "in",
        "content":{
            "field": "cases.project.primary_site",
            "value": ["Pancreas"]
            }
        },
        {
        "op": "in",
        "content":{
            "field": "files.analysis.workflow_type",
            "value": ["HTSeq - FPKM"]
            }
        }
    ]
}

# Here a GET is used, so the filter parameters should be passed as a JSON string.

params = {
    "filters": json.dumps(filters),
    "fields": "file_id",
    "format": "JSON",
    "size": "1000"
    }

response = requests.get(files_endpt, params = params)

file_uuid_list = []

# This step populates the download list with the file_ids from the previous query
for file_entry in json.loads(response.content.decode("utf-8"))["data"]["hits"]:
    file_uuid_list.append(file_entry["file_id"])

data_endpt = "https://api.gdc.cancer.gov/data"

params = {"ids": file_uuid_list}

response = requests.post(data_endpt, data = json.dumps(params), headers = {"Content-Type": "application/json"})

response_head_cd = response.headers["Content-Disposition"]


In [84]:
len(file_uuid_list)

182

In [85]:
file_name = '../data/TCGA/api/TCGA_PAAD_' + re.findall("filename=(.+)", response_head_cd)[0]

In [86]:
with open(file_name, "wb") as output_file:
    output_file.write(response.content)

In [87]:
!gunzip ../data/TCGA/api/TCGA_PAAD_gdc_download_20191207_012349.911695.tar.gz

gzip: ../data/TCGA/api/TCGA_PAAD_gdc_download_20191207_012349.911695.tar.gz: No such file or directory
